In [48]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
import numpy

In [49]:
data = pd.read_csv('./CSVs/DODFCorpus_contratos_licitacoes_v1.csv', dtype=str)

In [3]:
data

,Unnamed: 0.1,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,0,0,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,modalidade_licitacao,1,anotador_50435513,432851,17,PREGAO ELETRONICO,12_22.11.2018-R21
1,1,1,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,numero_licitacao,2,anotador_50435513,432872,8,116/2018,12_22.11.2018-R21
2,2,2,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,valor_estimado_contratacao,6,anotador_50435513,433432,10,"767.907,37",12_22.11.2018-R21
3,3,3,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,data_abertura_licitacao,7,anotador_50435513,433599,10,20/11/2018,12_22.11.2018-R21
4,4,4,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,sistema_compras,8,anotador_50435513,433716,21,www.compras.df.gov.br,12_22.11.2018-R21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40990,41054,41054,638,REL_SUSPENSAO_LICITACAO,R7,anotador_14379604,processo_gdf,258,anotador_14379604,210132,22,04012-00000112/2022-79,638-R7
40991,41055,41055,638,REL_SUSPENSAO_LICITACAO,R7,anotador_14379604,orgao_licitante,255,anotador_14379604,210158,63,Secretaria de Estado de Trabalho do Distrito F...,638-R7
40992,41056,41056,638,REL_SUSPENSAO_LICITACAO,R7,anotador_14379604,objeto_licitacao,3902,anotador_14379604,210322,367,Registro de Preços para contratação de empresa...,638-R7
40993,41057,41057,638,REL_SUSPENSAO_LICITACAO,R7,anotador_14379604,orgao_licitante,3206,anotador_14379604,210631,58,Secretaria de Estado do Trabalho Distrito Fede...,638-R7


In [50]:
#Insere um espaço entre as entidade e :
import re
def correct_space_before_numeric_entities(string):
    result = re.sub(r'([A-Za-z]:)[0-9]', r'\1 ', string)
    result = result.replace("\n", " ")
    return result

In [51]:

#Preprocessa o dataset para ajustar o casos de entidade e : sem espaço
#df['a']=df['a'].map(func)
# #data['texto']= data['texto'].apply(result)
data['texto']= data['texto'].map(correct_space_before_numeric_entities)


In [6]:
data

,Unnamed: 0.1,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,0,0,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,modalidade_licitacao,1,anotador_50435513,432851,17,PREGAO ELETRONICO,12_22.11.2018-R21
1,1,1,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,numero_licitacao,2,anotador_50435513,432872,8,116/2018,12_22.11.2018-R21
2,2,2,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,valor_estimado_contratacao,6,anotador_50435513,433432,10,"767.907,37",12_22.11.2018-R21
3,3,3,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,data_abertura_licitacao,7,anotador_50435513,433599,10,20/11/2018,12_22.11.2018-R21
4,4,4,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_50435513,sistema_compras,8,anotador_50435513,433716,21,www.compras.df.gov.br,12_22.11.2018-R21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40990,41054,41054,638,REL_SUSPENSAO_LICITACAO,R7,anotador_14379604,processo_gdf,258,anotador_14379604,210132,22,04012-00000112/2022-79,638-R7
40991,41055,41055,638,REL_SUSPENSAO_LICITACAO,R7,anotador_14379604,orgao_licitante,255,anotador_14379604,210158,63,Secretaria de Estado de Trabalho do Distrito F...,638-R7
40992,41056,41056,638,REL_SUSPENSAO_LICITACAO,R7,anotador_14379604,objeto_licitacao,3902,anotador_14379604,210322,367,Registro de Preços para contratação de empresa...,638-R7
40993,41057,41057,638,REL_SUSPENSAO_LICITACAO,R7,anotador_14379604,orgao_licitante,3206,anotador_14379604,210631,58,Secretaria de Estado do Trabalho Distrito Fede...,638-R7


In [52]:
data['id_ato'] = data['id_dodf'] + '-' + data['id_rel']
data

,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,0,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,modalidade_licitacao,1,anotador_22346437,432851,17,PREGAO ELETRONICO,12_22.11.2018-R21
1,1,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,numero_licitacao,2,anotador_22346437,432872,8,116/2018,12_22.11.2018-R21
2,2,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,valor_estimado_contratacao,6,anotador_22346437,433432,10,"767.907,37",12_22.11.2018-R21
3,3,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,data_abertura_licitacao,7,anotador_22346437,433599,10,20/11/2018,12_22.11.2018-R21
4,4,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,sistema_compras,8,anotador_22346437,433716,21,www.compras.df.gov.br,12_22.11.2018-R21
...,...,...,...,...,...,...,...,...,...,...,...,...
41036,41051,638,REL_SUSPENSAO_LICITACAO,R7,anotador_32161579,processo_gdf,258,anotador_32161579,210132,22,04012-00000112/2022-79,638-R7
41037,41052,638,REL_SUSPENSAO_LICITACAO,R7,anotador_32161579,orgao_licitante,255,anotador_32161579,210158,63,Secretaria de Estado de Trabalho do Distrito F...,638-R7
41038,41053,638,REL_SUSPENSAO_LICITACAO,R7,anotador_32161579,objeto_licitacao,3902,anotador_32161579,210322,367,Registro de Preços para contratação de empresa...,638-R7
41039,41054,638,REL_SUSPENSAO_LICITACAO,R7,anotador_32161579,orgao_licitante,3206,anotador_32161579,210631,58,Secretaria de Estado do Trabalho Distrito Fede...,638-R7


In [53]:
result = data.groupby('tipo_rel')['id_ato'].nunique()
print(result.sort_values(ascending=False))

tipo_rel
REL_EXTRATO_CONTRATO        1542
REL_ADITAMENTO_CONTRATO     1537
REL_AVISO_LICITACAO          638
REL_SUSPENSAO_LICITACAO       68
REL_ANUL_REVOG_LICITACAO      46
REL_EXTRATO_CONVENIO          24
Name: id_ato, dtype: int64


In [54]:
tipo_ato = data.loc[data['tipo_rel'] == 'REL_EXTRATO_CONTRATO'].reset_index(drop=True)
tipo_ato.to_csv("./CSVs/REL_EXTRATO_CONTRATO_COMPLETO_V1.csv")
tipo_ato

,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,179,12_22.11.2018,REL_EXTRATO_CONTRATO,R1,anotador_73210084,numero_contrato,355,anotador_73210084,556566,7,45/2018,12_22.11.2018-R1
1,180,12_22.11.2018,REL_EXTRATO_CONTRATO,R1,anotador_73210084,processo_gdf,354,anotador_73210084,556584,22,00070-00016600/2018-76,12_22.11.2018-R1
2,181,12_22.11.2018,REL_EXTRATO_CONTRATO,R1,anotador_73210084,orgao_contratante,356,anotador_73210084,556617,9,SEAGRI/DF,12_22.11.2018-R1
3,182,12_22.11.2018,REL_EXTRATO_CONTRATO,R1,anotador_73210084,entidade_contratada,357,anotador_73210084,556630,56,CIMAG COMERCIO DE IMPLEMENTOS E MAQUINAS AGRI...,12_22.11.2018-R1
4,183,12_22.11.2018,REL_EXTRATO_CONTRATO,R1,anotador_73210084,valor_contrato,359,anotador_73210084,557492,9,"39.600,00",12_22.11.2018-R1
...,...,...,...,...,...,...,...,...,...,...,...,...
20745,41043,638,REL_EXTRATO_CONTRATO,R85,anotador_73210084,valor_contrato,1184,anotador_73210084,209063,10,"547.811,64",638-R85
20746,41044,638,REL_EXTRATO_CONTRATO,R85,anotador_73210084,programa_trabalho,1188,anotador_73210084,209204,21,08.244.6228.4232.0002,638-R85
20747,41045,638,REL_EXTRATO_CONTRATO,R85,anotador_73210084,natureza_despesa,1189,anotador_73210084,209230,11,33.90.39.81,638-R85
20748,41046,638,REL_EXTRATO_CONTRATO,R85,anotador_73210084,nota_empenho,1191,anotador_73210084,209283,11,2022NE00130,638-R85


In [10]:

result = tipo_ato.groupby('tipo_ent')['id_ato'].nunique()
#print(result)
#print(result.sort_values(ascending=False))


In [11]:
#apenas para rodar o iob com spacy
#cria um novo dataset com as 2000 primeiras linhas
#csv_reader = pd.read_csv("/home/lucelia_vieira/Experimentos/ner/CSVs/REL_EXTRATO_CONTRATO_COMPLETO.csv", nrows=200)

In [55]:
#cria um novo dataset apenas com o tipo de entidade EXTATO_CONTRATO (TEXTO)
tipo_ato1 = tipo_ato.loc[tipo_ato['tipo_ent'] == 'EXTRATO_CONTRATO'].reset_index(drop=True)
tipo_ato1


,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,194,12_22.11.2018,REL_EXTRATO_CONTRATO,R1,anotador_73210084,EXTRATO_CONTRATO,448,anotador_73210084,556520,1834,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,12_22.11.2018-R1
1,210,12_22.11.2018,REL_EXTRATO_CONTRATO,R2,anotador_73210084,EXTRATO_CONTRATO,450,anotador_73210084,558353,1847,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,12_22.11.2018-R2
2,240,12_22.11.2018,REL_EXTRATO_CONTRATO,R3,anotador_73210084,EXTRATO_CONTRATO,399,anotador_73210084,572972,1187,EXTRATO DO CONTRATO No 32/2018 AQUISICAO DE BE...,12_22.11.2018-R3
3,297,12_22.11.2018,REL_EXTRATO_CONTRATO,R4,anotador_73210084,EXTRATO_CONTRATO,453,anotador_73210084,599347,2060,EXTRATO DE CONTRATO CONTRATO No 9020. ASSINATU...,12_22.11.2018-R4
4,371,12_22.11.2018,REL_EXTRATO_CONTRATO,R7,anotador_73210084,EXTRATO_CONTRATO,460,anotador_73210084,611856,1236,"EXTRATO DO CONTRATO No 01/2018, NOS TERMOS DO ...",12_22.11.2018-R7
...,...,...,...,...,...,...,...,...,...,...,...,...
1537,40929,638,REL_EXTRATO_CONTRATO,R81,anotador_73210084,EXTRATO_CONTRATO,1824,anotador_73210084,178278,1049,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,638-R81
1538,40939,638,REL_EXTRATO_CONTRATO,R82,anotador_73210084,EXTRATO_CONTRATO,1815,anotador_73210084,179328,1822,EXTRATO DO CONTRATO DE EXECUÇÃO DE OBRAS PARA ...,638-R82
1539,41009,638,REL_EXTRATO_CONTRATO,R83,anotador_73210084,EXTRATO_CONTRATO,1220,anotador_73210084,205946,1122,EXTRATO DO CONTRATO Nº 45.926/2022 Processo: 0...,638-R83
1540,41023,638,REL_EXTRATO_CONTRATO,R84,anotador_73210084,EXTRATO_CONTRATO,1206,anotador_73210084,207115,1267,EXTRATO DO CONTRATO Nº 45832/2022 Processo: 00...,638-R84


In [56]:
#criando novo dataframe somente com os textos
df = tipo_ato1['texto'].reset_index(drop=True)
df

0       EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...
1       EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...
2       EXTRATO DO CONTRATO No 32/2018 AQUISICAO DE BE...
3       EXTRATO DE CONTRATO CONTRATO No 9020. ASSINATU...
4       EXTRATO DO CONTRATO No 01/2018, NOS TERMOS DO ...
                              ...                        
1537    EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...
1538    EXTRATO DO CONTRATO DE EXECUÇÃO DE OBRAS PARA ...
1539    EXTRATO DO CONTRATO Nº 45.926/2022 Processo: 0...
1540    EXTRATO DO CONTRATO Nº 45832/2022 Processo: 00...
1541    EXTRATO DO CONTRATO Nº 45927/2022 Processo: 00...
Name: texto, Length: 1542, dtype: object

In [14]:
#df.to_csv("./CSVs/REL_EXTRATO_CONTRATO_COMPLETO_TEXTO_V1.csv")

In [15]:
#df = ["EXTRATO DO CONTRATO No 11/2019 PROCESSO: 00390-00003006/2019-92. ESPECIE: Contrato de Aquisicao de Bens pelo Distrito Federal no 11/2019. PARTES: Secretaria de Estado de Desenvolvimento Urbano e Habitacao do Distrito Federal - SEDUH/DF e INDUSTRIA DE AGUA MINERAL IBIA LTDA. DO OBJETO: Aquisicao de agua potavel de mesa sem gas e garrafao retornavel, destinados ao atendimento desta pasta, conforme demais especiGcacoes e condicoes estabelecidas no Termo de Referencia Anexo I do Edital de Pregao Eletronico no 0058/2018 - SCG/SEPLAG, da Proposta da empresa INDUSTRIA DE AGUA MINERAL IBIA LTDA de 11/06/2018 e da Lei no 8.666, de 21 de junho de 1993, que passam a integrar o presente Termo. DO VALOR: R$ 11.661,00 (onze mil, seiscentos e sessenta e um reais) conforme Nota de Empenho no 2019NE00180, emitida em 18/07/2019. Dotacao Orcamentaria P.T 15.122.6001.8517.0131 - Natureza da Despesa: 3390.30 - Fonte de Recursos: 100. DO PRAZO DE VIGENCIA: O Contrato tera vigencia de 06 (seis) meses a contar do dia 07/08/2019. DATA DE ASSINATURA: 22/07/2019. SIGNATARIOS: Pelo Distrito Federal: Mateus Leandro de Oliveira, na qualidade de Secretario de Estado. Pela Contratada: Eduardo Barros de Queiroz Rodrigues, na qualidade de Socio Administrador.", "EXTRATO DO CONTRATO Nº 06/2021 Processo: 00139-00001275/2021-00. Partes: DF - ADMINISTRAÇÃO REGIONAL DO CRUZEIRO X COMERCIAL DE ALIMENTOS PARK SUL. Objeto: O presente contrato tem por objeto a contratação de empresa especializada na área de alimentação para confecção e fornecimento de 50 KG DE BOLO CONFEITADO TIPO ANIVERSÁRIO COM MASSA PÃO DE LÓ COM RECHEIO DE BABA DE MOÇA, BEIJINHO OU DOCE DE LEITE E COBERTURA DE CHANTILLY DECORADO COM ACABAMENTO, no evento de comemoração ao aniversário de 62 anos do Cruzeiro no dia 30/11/2021. Valor do Contrato: R$ 2.000,00 (Dois Mil Reais). Dotação orçamentária: Unidade Orçamentária: 190113. Programa de Trabalho: 13.392.6219.3678.0136 . Natureza da Despesa: 339039. Fonte de Recursos: 120. Nota de Empenho: 2021NE00259. Valor do Empenho Inicial: R$ 2.000,00 (Dois Mil Reais), emitida em 26/11/2021 sob o evento 400091, na Modalidade 1   Ordinário. Fundamento Legal: Art. 24 da Lei nº 8.666/1993. Data da Assinatura: 29/11/2021. Signatários: Pelo DISTRITO FEDERAL: LUIZ EDUARDO GOMES DE PAULA PESSOA, na qualidade de Administrador Regional do Cruzeiro; pela COMERCIAL DE ALIMENTOS PARK SUL: GIVANILDES DE AGUIAR MORAIS, na qualidade de Gerente Geral."] 

In [57]:
# CODIGO RESPONSAVEL PELA LIMPEZA DO TEXTO DA BASE OURO
import re

df1 =[]

for i in df:
    i = i.replace("\n", " ")
    i = re.sub('xx[a-z]{1,10}', '', i)
    #i = i.replace(r"xx[a-z]{1,10}","")
    aux = ' '.join(i.split())
    #print(aux)
    df1.append(aux)


In [17]:
len(df1)

1538

In [18]:
""" ## Carrega o Teste para aplicação da LB direto
df_sk = pd.read_csv('./CSVs/Extrato_Contrato_Split_4_x_test_texto.csv', dtype=str)
df_sk """

" ## Carrega o Teste para aplicação da LB direto\ndf_sk = pd.read_csv('./CSVs/Extrato_Contrato_Split_4_x_test_texto.csv', dtype=str)\ndf_sk "

In [19]:
""" #criando novo dataframe de test somente com os textos
df_sk = df_sk['Texto'].reset_index(drop=True)
df_sk """

" #criando novo dataframe de test somente com os textos\ndf_sk = df_sk['Texto'].reset_index(drop=True)\ndf_sk "

In [58]:
# return_df=False para retornar atos e labels, ou  return_df=True para retornar dataset
from iob_transformer import iob_transformer
iob = iob_transformer('id_ato','texto','tipo_ent', keep_punctuation=False, return_df=False)

In [59]:
atos, labels = iob.transform(tipo_ato)

In [ ]:
len(atos)

1538

In [60]:
# CODIGO DE APLICACAO DA SUPERVISAO FRACA
import dodfSkweak_v1_2
skw = dodfSkweak_v1_2.SkweakContratos(df1)
skw.apply_label_functions()
skw.train_HMM_Dodf()
#data_label=skw.get_hmm_dataframe()


Starting iteration 1
Number of processed documents: 1000
Finished E-step with 1542 documents
Starting iteration 2


         1      -79795.6639             +nan


Number of processed documents: 1000
Finished E-step with 1542 documents
Starting iteration 3


         2      -77817.8681       +1977.7958


Number of processed documents: 1000
Finished E-step with 1542 documents
Starting iteration 4


         3      -77757.2023         +60.6658


Number of processed documents: 1000
Finished E-step with 1542 documents


         4      -77741.0209         +16.1814


Write to ./data/reuters_small.spacy...done


In [ ]:
#data_label

In [ ]:
# converting to CSV file
#data_label.to_csv("./CSVs/base_rotulada_supervisao_fraca.csv")

In [61]:
corpus = skw.get_IOB()

In [ ]:
#corpus

In [62]:
termos_skweak = []
debug=set()
labels_skweak = []
for ato in corpus:
    x = []
    y = []
    for termo in ato:
        termo_label = termo.split()
        x.append(termo_label[0])
        y.append(termo_label[1])
        debug.add(termo_label[1])
    termos_skweak.append(x)
    labels_skweak.append(y)

In [ ]:
debug

{'B-data_assinatura_contrato',
 'B-natureza_despesa',
 'B-nota_empenho',
 'B-numero_contrato',
 'B-processo_gdf',
 'B-programa_trabalho',
 'B-unidade_orcamentaria',
 'B-valor_contrato',
 'I-data_assinatura_contrato',
 'I-natureza_despesa',
 'I-nota_empenho',
 'I-numero_contrato',
 'I-processo_gdf',
 'I-programa_trabalho',
 'I-unidade_orcamentaria',
 'O'}

In [ ]:
len(labels_skweak)

1538

In [ ]:
len(termos_skweak)

1538

In [ ]:
#labels_skweak

In [63]:
termos_skweak_list = []
for i in termos_skweak:
   if not isinstance(i, list):
      termos_skweak_list.append(i)
   else:
      for j in i:
        termos_skweak_list.append(j)

len(termos_skweak_list)

368085

In [ ]:
#termos_skweak_list

In [64]:
labels_skweak_list = []
for i in labels_skweak:
   if not isinstance(i, list):
      labels_skweak_list.append(i)
   else:
      for j in i:
        labels_skweak_list.append(j)

len(labels_skweak_list)

368085

In [ ]:
#labels_skweak_list

In [65]:
labels_list = []
for i in labels:
   if not isinstance(i, list):
      labels_list.append(i)
   else:
      for j in i:
         labels_list.append(j)
len(labels_list)

368085

In [ ]:
#labels_list

In [66]:
atos_list = []
for i in atos:
   if not isinstance(i, list):
      atos_list.append(i)
   else:
      for j in i:
         atos_list.append(j)
len(atos_list)

368085

In [ ]:
corpus_skweak_list = []
for i in corpus:
   if not isinstance(i, list):
      corpus_skweak_list.append(i)
   else:
      for j in i:
         corpus_skweak_list.append(j)
#print(corpus_skweak_list)
len(corpus_skweak_list)

367495

In [ ]:
#corpus_skweak_list

In [67]:

resultados = []
for ato_sk_list , label_sk_list, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  atos_list, labels_list):
    tupla=(ato_sk_list , label_sk_list, ato_gold, label_gold)
    resultados.append(tupla)
    if ato_sk_list != ato_gold:
        print(ato_sk_list, ato_gold)
#for item in resultados:
#    print(item)

In [31]:
#labels_skweak = labels
numero_contrato = 0
numero_contrato_total = 0
B_numero_contrato = 0
numero_contrato_gold=[]

data_assinatura_contrato = 0
data_assinatura_contrato_total = 0
B_data_assinatura_contrato = 0
data_assinatura_contrato_gold = []

unidade_orcamentaria = 0
unidade_orcamentaria_total = 0
B_unidade_orcamentaria = 0
unidade_orcamentaria_gold = []
teste=[]

programa_trabalho = 0
programa_trabalho_total = 0
B_programa_trabalho = 0
programa_trabalho_gold = []

#for text_sk, label_skweak, text_gold,  label_gold in zip(termos_skweak, labels_skweak, atos, labels):
for ato_sk , label_sk, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  atos_list, labels_list):    
    #for text_skw, termo_skweak, text_ato, termo_gold in zip(text_sk, label_skweak, text_gold,  label_gold ):
       
       #print("termo swkeak", termo_skweak) 
       if label_gold == "B-numero_contrato":
           #print("Teste")
           numero_contrato_total+=1
           B_numero_contrato=label_gold
           if label_gold == label_sk:
               numero_contrato+=1
           else:
               numero_contrato_gold.append(ato_gold)

       elif label_gold == "B-data_assinatura_contrato":
                
            data_assinatura_contrato_total+=1
            B_data_assinatura_contrato=label_gold
            if label_gold == label_sk:
                data_assinatura_contrato+=1
            else:
                data_assinatura_contrato_gold.append(ato_gold)

       elif label_gold == "B-unidade_orcamentaria":
                
            unidade_orcamentaria_total+=1
            B_unidade_orcamentaria=label_gold
            if label_gold == label_sk:
                unidade_orcamentaria+=1
                
            else:
                unidade_orcamentaria_gold.append(ato_gold) 
                 
       elif label_gold == "B-programa_trabalho":
                
            programa_trabalho_total+=1
            B_programa_trabalho=label_gold
            if label_gold == label_sk:
                programa_trabalho+=1
            else:
                programa_trabalho_gold.append(ato_gold)                
print("Tipo da entidade             Total Entidade Gold                 Total Entidade Skweak")                
print("B-numero_contrato",              "   ",numero_contrato_total,         "   ",numero_contrato)#,"      ","       ",numero_contrato_gold)
print("B-data_assinatura_contrato"," ",data_assinatura_contrato_total,"   ",data_assinatura_contrato)#,"      ",data_assinatura_contrato_gold)
print("B-unidade_orcamentaria",      "  ",unidade_orcamentaria_total,    "   ",unidade_orcamentaria)#,"          ",unidade_orcamentaria_gold)
print( "B-programa_trabalho",       "  ",programa_trabalho_total,       "   ",programa_trabalho)#,"             ",programa_trabalho_gold)

#print(programa_trabalho_gold)

Tipo da entidade             Total Entidade Gold                 Total Entidade Skweak
B-numero_contrato     1680     1462
B-data_assinatura_contrato   963     896
B-unidade_orcamentaria    778     747
B-programa_trabalho    1103     1040


In [32]:
#supervisão fraca
x=termos_skweak
y=labels_skweak

In [33]:
#gold
x1=atos
y1=labels

In [34]:
#Split da Supervisão Fraca
from sklearn.model_selection import train_test_split
# 80% treino, 20% teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)
print( 'x_train',len(x_train),'\n','x_test',len(x_test),'\n','total atos',len(x))

x_train 1230 
 x_test 308 
 total atos 1538


In [35]:
#Carrega os atos do dataset de teste da base ouro
#https://pt.stackoverflow.com/questions/232208/ler-uma-lista-de-listas-em-txt-python-3
import pickle
with open("./CSVs/Extrato_Contrato_Split_V1_0_x_test.txt", "rb") as file:
    gold_x_test = pickle.load(file)

In [36]:
#Cria uma Lista com o x_test
gold_x_test_list = []
for i in gold_x_test:
   if not isinstance(i, list):
      gold_x_test_list.append(i)
   else:
      for j in i:
        gold_x_test_list.append(j)

len(gold_x_test_list)

72220

In [37]:
#Carrega os labels do dataset de teste da base ouro
import pickle
with open("./CSVs/Extrato_Contrato_Split_V1_0_y_test.txt", "rb") as file:
    gold_y_test = pickle.load(file)

In [38]:
gold_y_test_list = []
for i in gold_y_test:
   if not isinstance(i, list):
      gold_y_test_list.append(i)
   else:
      for j in i:
       gold_y_test_list.append(j)

len(gold_y_test_list)

72220

CRF

In [39]:
#X_train

def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x_train)):
    x_train[i] = get_features(x_train[i])

In [40]:
#X_test
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        x_test = []
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(gold_x_test)):
    gold_x_test[i] = get_features(gold_x_test[i])

In [41]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=10,
    c2=0.1,
    max_iterations=100,
    #max_iterations=50,
    
    all_possible_transitions=False,
    all_possible_states=True
)

#crf.fit(x_train, y_train)

try:
    #crf.fit(x_train[:200], y_train[:200])
    crf.fit(x_train, y_train)
except AttributeError:
    pass


In [ ]:
crf.classes_

['O',
 'B-numero_contrato',
 'B-processo_gdf',
 'I-processo_gdf',
 'B-nota_empenho',
 'B-unidade_orcamentaria',
 'B-programa_trabalho',
 'B-natureza_despesa',
 'B-data_assinatura_contrato',
 'B-valor_contrato',
 'I-nota_empenho',
 'I-data_assinatura_contrato',
 'I-numero_contrato',
 'I-natureza_despesa',
 'I-programa_trabalho',
 'I-unidade_orcamentaria']

In [42]:
#Rotulação baseada no modelo treinado com supervisão fraca
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(gold_x_test)
metrics.flat_f1_score(gold_y_test, y_pred, average='weighted', labels=classes)

/home/lucelia_vieira/experiments/members/lucelia/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.7542472606516447

In [45]:
y_pred_list = []
for i in y_pred:
   if not isinstance(i, list):
      y_pred_list.append(i)
   else:
      for j in i:
       y_pred_list.append(j)

len(y_pred_list)

72220

In [46]:
resultados = []
for ato_gold, label_gold, label_pred in zip(gold_x_test_list, gold_y_test_list, y_pred_list):
    tupla=(ato_gold,  label_gold, label_pred)
    resultados.append(tupla)
    #if gold_y_test_list != y_pred:
    #    print(gold_y_test_list, y_pred)
#print(resultados)
for item in resultados:
    print(item)

('EXTRATO', 'O', 'O')
('DO', 'O', 'O')
('CONTRATO', 'O', 'O')
('DE', 'O', 'O')
('PRESTAÇÃO', 'O', 'O')
('DE', 'O', 'O')
('SERVIÇOS', 'O', 'O')
('Nº', 'O', 'O')
('25/2021', 'B-numero_contrato', 'B-numero_contrato')
('Processo', 'O', 'O')
(':', 'O', 'O')
('00094-00005321/', 'B-processo_gdf', 'B-processo_gdf')
('2019-81', 'I-processo_gdf', 'I-processo_gdf')
('.', 'O', 'O')
('PARTES', 'O', 'O')
(':', 'O', 'O')
('SERVIÇO', 'B-orgao_contratante', 'O')
('DE', 'I-orgao_contratante', 'O')
('LIMPEZA', 'I-orgao_contratante', 'O')
('URBANA', 'I-orgao_contratante', 'O')
('DO', 'I-orgao_contratante', 'O')
('DISTRITO', 'I-orgao_contratante', 'O')
('FEDERAL', 'I-orgao_contratante', 'O')
('-', 'I-orgao_contratante', 'O')
('SLU', 'B-orgao_contratante', 'O')
('/', 'I-orgao_contratante', 'O')
('DF', 'I-orgao_contratante', 'O')
('e', 'O', 'O')
('a', 'O', 'O')
('empresa', 'O', 'O')
('CONÁGUA', 'B-entidade_contratada', 'O')
('AMBIENTAL', 'I-entidade_contratada', 'O')
('LTDA', 'I-entidade_contratada', 'O')
('

In [47]:
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.metrics import f1_score

report = classification_report(gold_y_test, y_pred, output_dict=False, mode='strict', scheme=IOB2)
print(report)
np.save("./Results/METRICA_EXTRATO_CONTRATO_SKWEAK_V1_0_sq.npy", report)

                          precision    recall  f1-score   support

cnpj_entidade_contratada       0.00      0.00      0.00        73
  cnpj_orgao_contratante       0.00      0.00      0.00        13
            codigo_siggo       0.00      0.00      0.00        34
data_assinatura_contrato       0.55      0.85      0.67       185
     entidade_contratada       0.00      0.00      0.00       323
           fonte_recurso       0.00      0.00      0.00       263
        natureza_despesa       1.00      0.61      0.76       222
        nome_responsavel       0.00      0.00      0.00        41
            nota_empenho       0.96      0.72      0.82       258
         numero_contrato       0.92      0.60      0.72       340
         objeto_contrato       0.00      0.00      0.00       304
       orgao_contratante       0.00      0.00      0.00       444
            processo_gdf       0.89      0.79      0.83       298
       programa_trabalho       0.57      0.52      0.54       221
    unida

/home/lucelia_vieira/experiments/members/lucelia/.venv/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
   gold_y_test, y_pred, labels=classes, digits=3, output_dict=False)
print(report)

/home/lucelia_vieira/experiments/members/lucelia/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucelia_vieira/experiments/members/lucelia/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

         B-numero_contrato      0.923     0.597     0.725       340
            B-processo_gdf      0.897     0.792     0.841       298
            I-processo_gdf      0.894     0.825     0.858       285
            B-nota_empenho      0.974     0.725     0.831       258
    B-unidade_orcamentaria      0.860     0.613     0.716       150
       B-programa_trabalho      0.599     0.548     0.572       221
        B-natureza_despesa      1.000     0.613     0.760       222
B-data_assinatura_contrato      0.547     0.854     0.667       185
          B-valor_contrato      0.795     0.887     0.839       381
            I-nota_empenho      0.000     0.000     0.000         4
I-data_assinatura_contrato      0.400     0.459     0.427        61
         I-numero_contrato      0.000     0.000     0.000        12
        I-natureza_despesa      0.000     0.000     0.000         3
       I-programa_trabalho      0.000     0.000

In [ ]:
#output_dict=True gerar a saída para gravar no results

import numpy as np
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
    gold_y_test, y_pred, labels=classes, digits=3, output_dict=True)
print(report)
np.save("./Results/METRICA_EXTRATO_CONTRATO_SKWEAK_V1_0.npy", report)

{'B-numero_contrato': {'precision': 0.8893805309734514, 'recall': 0.5911764705882353, 'f1-score': 0.7102473498233216, 'support': 340}, 'B-processo_gdf': {'precision': 0.9098360655737705, 'recall': 0.7449664429530202, 'f1-score': 0.8191881918819188, 'support': 298}, 'I-processo_gdf': {'precision': 0.9057377049180327, 'recall': 0.775438596491228, 'f1-score': 0.8355387523629488, 'support': 285}, 'B-nota_empenho': {'precision': 0.9740932642487047, 'recall': 0.7286821705426356, 'f1-score': 0.8337028824833703, 'support': 258}, 'B-unidade_orcamentaria': {'precision': 0.5068493150684932, 'recall': 0.49333333333333335, 'f1-score': 0.5, 'support': 150}, 'B-programa_trabalho': {'precision': 0.7441860465116279, 'recall': 0.4343891402714932, 'f1-score': 0.5485714285714286, 'support': 221}, 'B-natureza_despesa': {'precision': 1.0, 'recall': 0.5900900900900901, 'f1-score': 0.7422096317280453, 'support': 222}, 'B-data_assinatura_contrato': {'precision': 0.48757763975155277, 'recall': 0.848648648648648

In [ ]:
# CODIGO DE APLICACAO DO MODELO CRF TREINADO

#import dodfCRF
#crf = dodfCRF.CRFContratos()
#crf.init_model_from_path("contratosFINAL.pkl")
#crfIOB = crf.model_predict(dados)